# How do we choose hyperpriors on our BayesianRidge fitting code?

## Background
When fitting a prior
We need to choose $\alpha_1$ and $\alpha_2$ to represent our underlying assumption about the intrinsic width of a spiral arm

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from astropy.visualization import hist
from scipy.special import gamma
from scipy.optimize import curve_fit
import scipy.special as sc
import lib.galaxy_utilities as gu

In [ ]:
all_annotations = [c['annotations'] for c in classifications]

In [ ]:
def gd(x, alpha, beta):
    return beta**alpha / gamma(alpha) * x**(alpha-1) * np.exp(-beta * x) * 1 / sc.gammaincc(alpha, beta * 0.1)

In [ ]:
all_annotations = [c['annotations'] for c in classifications]
def grab_slider_spread(annotation):
    try:
        return [float(j['details'][1]['value']) for j in annotation[3]['value'][0]['value']]
    except (KeyError, IndexError):
        return []
width_sliders = np.array([j for i in all_annotations for j in grab_slider_spread(i)])
default_mask = width_sliders == 1.0
min_max_mask = (width_sliders == 0.0) ^ (width_sliders == 2.0)
width_sliders_filtered = width_sliders[~(default_mask ^ min_max_mask)]

sigmas = 1 / (1**2 * 5 * width_sliders_filtered)
print(min(sigmas), max(sigmas))

y, X_, _ = hist(sigmas, bins='knuth', alpha=1, histtype='step', density=True)
X = (X_[1:] + X_[:-1])/2

popt, _ = curve_fit(gd, X, y, (1, 1))
plt.plot(X, gd(X, *popt))
print('Semi-truncated gamma parameters', popt)